# Actividad 3:
# Comparación de métodos de Boosting y Bagging en predicción de ingresos

## Objetivo
Aplicar y comparar algoritmos de boosting y bagging sobre datos reales, evaluando su rendimiento mediante precisión y matriz de confusión, e interpretando los resultados para fundamentar decisiones técnicas.

**Datasets utilizados:**  
`Adult Income`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos y entrenamiento de modelos:**

Se utilizaron 3 datasets para evaluar 3 técnicas avanzadas de regresión:  

   - Se utilizó el dataset **Adult Incoome** desde `fetch_openml`.
   - Identificación de valores faltantes (?), tipos de variables y distribución de clases.
   - Limpieza de datos reemplazando o eliminando valores nulos según su proporción.
   - Codificación ordinal de variables categóricas para estandarizar la entrada a todos los modelos.
   - División estratificada del dataset en entrenamiento y prueba (80/20).

2. **Evaluación y análisis:**
   - Modelos utilizados:
     - Boosting: AdaBoost, XGBoost, LightGBM, CatBoost.
     - Bagging: Random Forest.

   - Optimización de modelos:
     - Uso de Optuna con 20 trials por modelo.
     - Validación cruzada (CV=3) y accuracy como métrica objetivo.
     - Entrenamiento de modelos en abse a los mejores parámetros.

   - Métricas empleadas:
     - accuracy_score
     - confusion_matrix
     - Tiempos de ejecución

3. **Visualización de resultados:**
  - Tabla resumen con métricas evaluadas para cada modelo.
  - Gráficos de barras comparativos para mostrar la comparación de accuracy y tiempos de ejecución entre los cinco modelos.
  - Matrices de confusión de cada modelo.

---

# 2. Configuración del entorno

--- 

In [ ]:
import pandas as pd
import numpy as np
import time
import optuna
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")

# Configuración visual y de entorno
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos y entrenamiento de modelos.

- **`cargar_y_analizar_datos()`** 
Carga el dataset Adult Income desde OpenML, analiza su estructura y limpia valores nulos.

- **`preprocesar_datos()`** 
Aplica imputación y codificación ordinal a las variables, y separa los datos en entrenamiento y prueba.

In [ ]:
def cargar_y_analizar_datos():
    """
    Carga y realiza un análisis exploratorio inicial del dataset 'Adult Income' desde OpenML.

    El flujo incluye:
    - Carga del dataset como DataFrame de pandas.
    - Impresión de las primeras filas y resumen informativo de las columnas.
    - Revisión del número de valores únicos por columna.
    - Reemplazo de caracteres '?' por valores nulos (NaN).
    - Revisión e imputación de valores faltantes:
        - Si una columna tiene <5% de valores nulos, se eliminan las filas correspondientes.
        - Si una columna tiene >=5% de valores nulos, se imputan con la moda de la columna.

    Returns:
        pd.DataFrame: DataFrame limpio y listo para preprocesamiento posterior.
    """
    print("Cargando datos...")
    df = fetch_openml("adult", version=2, as_frame=True).frame
    print("\nInformación del dataset:")
    print(df.info())

    # Reemplazar '?' con NaN
    df.replace('?', np.nan, inplace=True)

    # Ver valores nulos
    print("\nPorcentaje de valores faltantes:")
    print(df.isnull().mean() * 100)

    # Eliminar columnas o filas según porcentaje de nulos
    for col in df.columns:
        missing = df[col].isnull().mean()
        if missing > 0:
            if missing < 0.05:
                df = df[df[col].notnull()]
            else:
                df[col] = df[col].fillna(df[col].mode()[0])

    return df

def preprocesar_datos(df):
    """
    Preprocesa el DataFrame del dataset Adult para preparar los datos para entrenamiento de modelos de ML.

    El flujo incluye:
    - Separación entre variables predictoras (X) y variable objetivo ('class').
    - Codificación de la variable objetivo (binaria) usando LabelEncoder.
    - Imputación de valores faltantes en X con la moda de cada columna.
    - Codificación ordinal de todas las variables categóricas para convertirlas a valores numéricos.
    - Validación de que no queden columnas categóricas sin codificar.
    - División de los datos en conjuntos de entrenamiento y prueba con estratificación.

    Args:
        df (pd.DataFrame): Dataset limpio con variables categóricas y numéricas.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: 
            X_train, X_test, y_train, y_test listos para entrenamiento.
    """
    df = df.copy()
    y = df['class']
    le = LabelEncoder()
    y = le.fit_transform(y)

    X = df.drop('class', axis=1)
    X = X.replace('?', np.nan)

    for col in X.columns:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mode()[0])

    cat_cols = X.select_dtypes(include=['object', 'category']).columns
    enc = OrdinalEncoder()
    X[cat_cols] = enc.fit_transform(X[cat_cols])

    assert X.select_dtypes(include=['object', 'category']).empty, "[ALERTA] Quedan columnas no numéricas en X"

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    return X_train, X_test, y_train, y_test

**Bloque 2:** Optimización, entrenamiento y evaluación de modelos de Boosting y Bagging.

- **`optimizar_modelo()`** 
Usa Optuna para encontrar los mejores hiperparámetros para un modelo dado.

- **`entrenar_modelo()`** 
Entrena el modelo con los hiperparámetros óptimos y mide el tiempo de entrenamiento.

- **`evaluar_modelo()`** 
Evalúa el modelo sobre los datos de prueba usando accuracy y matriz de confusión.

In [ ]:
def optimizar_modelo(nombre_modelo, X_train, y_train):
    """
    Optimiza los hiperparámetros de un modelo de clasificación utilizando Optuna y validación cruzada.

    Dependiendo del nombre del modelo, define un espacio de búsqueda de hiperparámetros específicos 
    y utiliza Optuna para encontrar la mejor combinación que maximice la precisión (accuracy).

    Modelos soportados:
    - AdaBoost
    - XGBoost
    - LightGBM
    - CatBoost
    - RandomForest

    Args:
        nombre_modelo (str): Nombre del modelo a optimizar. Debe ser uno de los modelos soportados.
        X_train (np.ndarray or pd.DataFrame): Conjunto de entrenamiento (features).
        y_train (np.ndarray or pd.Series): Conjunto de entrenamiento (target).

    Returns:
        dict: Diccionario con los mejores hiperparámetros encontrados por Optuna.
    """
    def objetivo(trial):
        if nombre_modelo == 'AdaBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
                'random_state': 42
            }
            modelo = AdaBoostClassifier(**params)

        elif nombre_modelo == 'XGBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'eval_metric': 'logloss',
                'random_state': 42
            }
            modelo = XGBClassifier(**params)

        elif nombre_modelo == 'LightGBM':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'random_state': 42
            }
            modelo = LGBMClassifier(**params, verbosity = -1)

        elif nombre_modelo == 'CatBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'depth': trial.suggest_int('depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'verbose': 0,
                'random_state': 42
            }
            modelo = CatBoostClassifier(**params)

        elif nombre_modelo == 'RandomForest':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'random_state': 42
            }
            modelo = RandomForestClassifier(**params)

        score = cross_val_score(modelo, X_train, y_train, cv=3, scoring='accuracy', error_score='raise').mean()
        return score

    estudio = optuna.create_study(direction='maximize')
    estudio.optimize(objetivo, n_trials=20, timeout=300)
    print(f"[INFO] Mejor conjunto de hiperparámetros para {nombre_modelo}: {estudio.best_params}")
    return estudio.best_params

def entrenar_modelo(nombre_modelo, params, X_train, y_train):
    """
    Entrena un modelo de clasificación utilizando los hiperparámetros especificados.

    Se selecciona e instancia el modelo correspondiente, se ajusta con los datos de entrenamiento,
    y se mide el tiempo total de entrenamiento.

    Args:
        nombre_modelo (str): Nombre del modelo a entrenar.
        params (dict): Diccionario con los hiperparámetros optimizados.
        X_train (np.ndarray or pd.DataFrame): Conjunto de entrenamiento (features).
        y_train (np.ndarray or pd.Series): Conjunto de entrenamiento (target).

    Returns:
        Tuple[object, float]: 
            - El modelo entrenado.
            - Tiempo de entrenamiento en segundos.
    """
    start = time.time()
    if nombre_modelo == 'AdaBoost':
        modelo = AdaBoostClassifier(**params)
    elif nombre_modelo == 'XGBoost':
        params['eval_metric'] = 'logloss'
        modelo = XGBClassifier(**params)
    elif nombre_modelo == 'LightGBM':
        modelo = LGBMClassifier(**params)
    elif nombre_modelo == 'CatBoost':
        modelo = CatBoostClassifier(**params, verbose=0)
    elif nombre_modelo == 'RandomForest':
        modelo = RandomForestClassifier(**params)

    modelo.fit(X_train, y_train)
    duracion = time.time() - start
    return modelo, duracion

def evaluar_modelo(modelo, X_test, y_test):
    """
    Evalúa un modelo de clasificación en el conjunto de prueba.

    Calcula el accuracy y la matriz de confusión a partir de las predicciones del modelo.

    Args:
        modelo (object): Modelo previamente entrenado con método `.predict()`.
        X_test (np.ndarray or pd.DataFrame): Conjunto de prueba (features).
        y_test (np.ndarray or pd.Series): Conjunto de prueba (target).

    Returns:
        Tuple[float, np.ndarray]: 
            - Accuracy del modelo en test.
            - Matriz de confusión (2x2).
    """
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    matriz = confusion_matrix(y_test, y_pred)
    return acc, matriz

**Bloque 3:** Visualización de resultados.

- **`graficar_resultados()`** 
Genera gráficos comparativos de accuracy y tiempo de entrenamiento para todos los modelos.

- **`graficar_matrices_confusion()`** 
Muestra matrices de confusión en formato de mapas de calor para cada modelo.

In [ ]:
def graficar_resultados(resultados):
    """
    Genera gráficos de barras para visualizar el rendimiento de distintos modelos de clasificación.

    Se generan tres gráficos:
    - Accuracy por modelo.
    - Tiempo de entrenamiento por modelo (en segundos).
    - Tiempo total por modelo (optimización + entrenamiento, en segundos).

    Args:
        resultados (dict): Diccionario donde las claves son nombres de modelos y los valores son listas con:
                           [accuracy, tiempo_entrenamiento, tiempo_optimizacion, tiempo_total].

    Returns:
        None. Muestra las figuras usando matplotlib.
    """
    df_resultados = pd.DataFrame(resultados).T.reset_index()
    df_resultados.columns = ['Modelo', 'Accuracy', 'Tiempo_Entrenamiento', 'Tiempo_Optimizacion', 'Tiempo_Total']

    fig, axs = plt.subplots(1, 3, figsize=(18, 5))  # Una fila, tres columnas

    sns.barplot(x='Modelo', y='Accuracy', data=df_resultados, ax=axs[0], color='Orange')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_title('Accuracy por Modelo')
    axs[0].tick_params(axis='x', rotation=45)

    sns.barplot(x='Modelo', y='Tiempo_Entrenamiento', data=df_resultados, ax=axs[1], color='Green')
    axs[1].set_ylabel('Tiempo (s)')
    axs[1].set_title('Tiempo de Entrenamiento')
    axs[1].tick_params(axis='x', rotation=45)

    sns.barplot(x='Modelo', y='Tiempo_Total', data=df_resultados, ax=axs[2], color='Blue')
    axs[2].set_ylabel('Tiempo (s)')
    axs[2].set_title('Tiempo Total (Opt + Entrenamiento)')
    axs[2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

def graficar_matrices_confusion(matrices, nombres_modelos):
    """
    Visualiza las matrices de confusión de distintos modelos como mapas de calor (heatmaps).

    Dibuja hasta 5 subplots organizados en una grilla (2 filas x 3 columnas), uno para cada modelo. 
    Si hay menos de 6 modelos, se deja vacío el subplot restante para mantener la simetría.

    Args:
        matrices (list of np.ndarray): Lista de matrices de confusión, una por modelo.
        nombres_modelos (list of str): Lista con los nombres de los modelos, en el mismo orden que las matrices.

    Returns:
        None. Muestra las figuras usando matplotlib.
    """
    fig = plt.figure(figsize=(15, 10))
    gs = GridSpec(2, 3, figure=fig)

    # Primera fila: 3 gráficos
    axs = [fig.add_subplot(gs[0, i]) for i in range(3)]
    # Segunda fila: 2 gráficos en las dos primeras columnas
    axs += [fig.add_subplot(gs[1, i]) for i in range(2)]

    for i, (nombre, matriz) in enumerate(zip(nombres_modelos, matrices)):
        sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues', ax=axs[i])
        axs[i].set_title(nombre)
        axs[i].set_xlabel('Predicción')
        axs[i].set_ylabel('Real')

    # Eliminar ejes del subplot vacío (última posición)
    fig.delaxes(fig.add_subplot(gs[1, 2]))

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución del código.

- **`main()`** 
Ejecuta todo el flujo: carga, preprocesamiento, optimización, entrenamiento, evaluación, visualización y selección del mejor modelo.

In [ ]:
def main():
    """
    Función principal que ejecuta todo el flujo del proyecto de clasificación con métodos de ensamble.

    Realiza las siguientes tareas:
    1. Carga y limpieza del dataset 'Adult Income' desde OpenML.
    2. Preprocesamiento de variables (imputación, codificación ordinal y división en train/test).
    3. Optimización de hiperparámetros para cinco modelos: AdaBoost, XGBoost, LightGBM, CatBoost y RandomForest.
    4. Medición del tiempo requerido para optimización y entrenamiento de cada modelo.
    5. Entrenamiento de cada modelo con los mejores hiperparámetros encontrados.
    6. Evaluación de cada modelo con métricas de precisión (accuracy) y matriz de confusión.
    7. Visualización comparativa del rendimiento de los modelos en términos de accuracy, tiempo de entrenamiento, optimización y total.
    8. Impresión de resultados finales y selección del mejor modelo para producción.

    Returns:
        None. Ejecuta todo el pipeline de principio a fin y muestra resultados por consola y gráficos.
    """
    df = cargar_y_analizar_datos()
    X_train, X_test, y_train, y_test = preprocesar_datos(df)

    modelos = ['AdaBoost', 'XGBoost', 'LightGBM', 'CatBoost', 'RandomForest']
    resultados = {}
    matrices = []

    for modelo in modelos:
        print(f"\nOptimizando hiperparámetros para {modelo}...")
        inicio_opt = time.time()
        params = optimizar_modelo(modelo, X_train, y_train)
        tiempo_opt = time.time() - inicio_opt

        print(f"Entrenando modelo {modelo}...")
        modelo_entrenado, tiempo_ent = entrenar_modelo(modelo, params, X_train, y_train)

        tiempo_total = tiempo_opt + tiempo_ent

        acc, matriz = evaluar_modelo(modelo_entrenado, X_test, y_test)

        resultados[modelo] = [acc, tiempo_ent, tiempo_opt, tiempo_total]
        matrices.append(matriz)

    # Prepara los datos para tabulate
    tabla = []
    for modelo, (acc, tiempo_ent, tiempo_opt, tiempo_total) in resultados.items():
        tabla.append([modelo, f"{acc:.4f}", f"{tiempo_ent:.2f} s", f"{tiempo_opt:.2f} s", f"{tiempo_total:.2f} s"])

    # Define los encabezados
    headers = ["Modelo", "Accuracy", "Tiempo Entrenamiento", "Tiempo Optimización", "Tiempo Total"]

    print("\nResultados finales:")
    print(tabulate(tabla, headers=headers, tablefmt="grid"))

    graficar_resultados(resultados)
    graficar_matrices_confusion(matrices, modelos)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == '__main__':
    main()

# Análisis de resultados y reflexiones finales.

---

>**NOTA**: Los resultados aca expuestos y analizados fueron sacados de una ejecución x, por lo que, al momento de ejecutar nuevamente el código los resultados podrían variar, principalmente en cuál es el mejor modelo, esto debido a que como se puede apreciar, tanto XGBoost como LightGBM y CatBoost tienen valores muy similares de accuracy. La única diferencia es que Catboost tiene un peor tiempo de ejecución, teniendo en cuenta los tiempos de optimización y de entrenamiento. En el caso de la ejecución usada en este análisis, XGBoost tuvo el mejor accuracy.

---

## Comparación de resultados entre modelos

### Comparación de accuracy:

- Los modelos de Boosting (XGBoost, LightGBM, CatBoost) alcanzan accuracies muy similares, en torno a 0.868, ligeramente superiores a los métodos clásicos como AdaBoost y RandomForest.
- Entre ellos, XGBoost es el que logró la mayor precisión, aunque por muy poco margen.

### Tiempos de optimización y entrenamiento:

- Aquí la diferencia es más notable: XGBoost y LightGBM fueron los mas rápidos, luego CatBoost, y finalmente los métodos tradicionales que tardan varios segundos.
- Esto indica que XGBoost y LightGBM no solo son precisos, sino también muy eficientes en términos de tiempo.

### Preprocesamiento y codificación:

- Se aplicó una codificación ordinal a las variables categóricas, compatible con todos los modelos.
- Modelos como CatBoost pueden manejar variables categóricas de forma nativa, pero se decidió codificar todas las variables para tener uniformidad y comparabilidad.

### Optimización hiperparamétrica:

- Se utilizó Optuna para buscar los mejores hiperparámetros, con 20 trials y validación cruzada, asegurando que los modelos estén optimizados para el dataset.
- Esto fortalece la validez de las comparaciones.

---

## ¿Cuál modelo se usaría en producción?

Basándonos en la combinación de alta precisión y rapidez en entrenamiento, **XGBoost** es el mejor candidato para producción en este caso.

### Justificación:

- Máxima precisión alcanzada: 0.8702, aunque por un margen pequeño, supera al resto de modelos.
- Alta eficiencia: Entrenamiento rápido, apenas por detrás de LightGBM, con excelente balance entre tiempo y rendimiento.
- Estabilidad y robustez: No presentó advertencias durante la optimización, a diferencia de LightGBM (e.g. "No further splits with positive gain").
- Amplia adopción en la industria: XGBoost es altamente probado, con buena documentación, soporte y escalabilidad.
- Flexible y configurable: Permite ajustes finos para tareas específicas y se integra fácilmente en entornos productivos.

### Comparación con otros:

- LightGBM es un poco peor en rendimiento, pero levemente mas rápido.
- CatBoost es potente y maneja categóricas sin codificación previa, pero aquí su tiempo es mayor y la ganancia de accuracy no justifica la mayor latencia.
- RandomForest y AdaBoost quedan rezagados en precisión y velocidad.

---

## Reflexiones finales

En este trabajo se comprobó que los modelos de Boosting, especialmente XGBoost y LightGBM, ofrecen un rendimiento superior en términos de precisión y tiempo de entrenamiento frente a métodos tradicionales como AdaBoost y Random Forest. La cuidadosa optimización de hiperparámetros mediante Optuna, junto con un preprocesamiento consistente de los datos, fue fundamental para lograr resultados robustos y comparables entre modelos.

Para aplicaciones en producción, es crucial considerar no solo la precisión sino también la eficiencia computacional y la escalabilidad. XGBoost destacó como la mejor opción en este caso, combinando alta exactitud con tiempos de entrenamiento muy bajos, lo que facilita su integración en pipelines reales donde la rapidez y la confiabilidad son claves. Además, este análisis resalta la importancia de adaptar el modelo al contexto y las necesidades específicas del proyecto.

### Consideraciones a tomar en cuenta sobre los datos

Este análisis parte del dataset Adult Income, con sus características específicas.
- En entornos reales, también se deben evaluar:
    - Robustez del modelo ante datos nuevos o no vistos.
    - Facilidad de integración y despliegue.
    - Interpretabilidad (LightGBM puede ser menos interpretable que RandomForest, pero esto puede mitigarse con técnicas como SHAP).
    - Costos computacionales y hardware disponible.